# NAUTILUS Navigator example

This example goes through the basic functionalities of the NAUTILUS Navigator method.

We will consider a simple 2D Pareto front which we will define next alongside the method itself. Both objectives are to be minimized.

Because of the nature of navigation based interactive optimization methods, the idea of NAUTILUS Navigator is best demonstrated using some graphical user interface. One such interface can be found [online](https://dash.misitano.xyz).

In [ ]:
import numpy as np 
from desdeo_mcdm.interactive.NautilusNavigator import NautilusNavigator

# half of a parabola to act as a Pareto front
f1 = np.linspace(1, 100, 50)
f2 = f1[::-1] ** 2

front = np.stack((f1, f2)).T
ideal = np.min(front, axis=0)
nadir = np.max(front, axis=0)

method = NautilusNavigator((front), ideal, nadir)

To start, we can invoke the `start` method.

In [ ]:
req_first = method.start()

print(req_first)
print(req_first.content.keys())

The returned object is a NautilusNavigatorRequest. The keys should give an idea of what the contents of the request are. We will explain most of the in this example.

At the moment, the `nadir`, `reachable_lb` and `reachable_ub` are most interesting to us. Navigation starts from the nadir and will proceed toward the Pareto optimal front enclosed between the limits defined in `reachable_lb` and `reachable_ub`.

To interact with the method, we must fill out the `response` member of `req`. Let's see the contents of the message in `req` next.

In [ ]:
print(req_first.content["message"])

We should define the required values and set them as keys of a dictionary. Before that, it is useful to see the bounds to know the currently feasible objective values.

In [ ]:
print(req_first.content["reachable_lb"])
print(req_first.content["reachable_ub"])

In [ ]:
reference_point = np.array([50, 6000])
go_to_previous = False
stop = False
speed = 1

response = dict(reference_point=reference_point, go_to_previous=False, stop=False, speed=1)

`go_to_previous` should be set to `False` unless we desire going to a previous point. `stop` should be `True` if we wish to stop, otherwise it should be `False`. `speed` is the speed of the navigation. It is not used internally in the method. To continue, we call `iterate` with suppliying the `req` object with a defined `response` attribute. We should get a new request as a return value.

In [ ]:
req_first.response = response
req_snd = method.iterate(req_first)

print(req_snd.content["reachable_lb"])
print(req_snd.content["reachable_ub"])

We see that the bounds have narrowed down as they should.

In reality, `iterate` should be called multiple times in succession with the same `response` contents. We can do this in a loop until the 30th step is computed, for example. NB: Steps are internally zero-index based.

In [ ]:
previous_requests = [req_first, req_snd]
req = req_snd
while method._step_number < 30:
    req.response = response
    req = method.iterate(req)

    previous_requests.append(req)

print(req.content["reachable_lb"])
print(req.content["reachable_ub"])
print(req.content["step_number"])

The region of reachable Pareto optimal solutions had narrowed down. Suppose now we wish to return to a previous step and change our preferences. Let's say, step 14.

In [ ]:
# fetch the 14th step saved previously
req_14 = previous_requests[13]
print(req_14.content["reachable_lb"])
print(req_14.content["reachable_ub"])
print(req_14.content["step_number"])

req_14.response["go_to_previous"] = True
req_14.response["reference_point"] = np.array([50, 5000])
new_response = req_14.response

When going to a previous point, the method assumes thath the state the method was in during that point is fully defined in the request object given to it when calling `iterate` with `go_to_previous` being `True`. This is why we saved the request previously in a list.

In [ ]:
req_14_new = method.iterate(req_14)
req = req_14_new

# remember to unser go_to_previous!
new_response["go_to_previous"] = False

# continue iterating for 16 steps
while method._step_number < 30:
    req.response = new_response
    req = method.iterate(req)

print("Old 30th step")
print(previous_requests[29].content["reachable_lb"])
print(previous_requests[29].content["reachable_ub"])
print(previous_requests[29].content["step_number"])

print("New 30th step")
print(req.content["reachable_lb"])
print(req.content["reachable_ub"])
print(req.content["step_number"])

We can see a difference in the limits when we changed the preference point.

To find the final solution, we can iterate till the end.

In [ ]:
while method._step_number < 100:
    req.response = new_response
    req = method.iterate(req)

print(req.content["reachable_idx"])

When finished navigating, the method will return the index of the reached solution based on the supplied Pareto front. It is assumed that if decision variables also exist for the problem, they are stored elwhere. The final index returned can then be used to find the corresponding decision variables to the found solution in objective space.